In [ ]:
#default_exp rest

In [ ]:
#export
import requests
import nifi_api as na
from nifi_api.endpoints import *
from nifi_api.tools import custom_response
import json 

# REST
> The NIFI REST API calls

## Processors

In [ ]:
#export 
class Processor:
    
    nifi_api = NIFI_API_PROCESSORS

    @classmethod
    def get(cls, processor_id):
        url = nifi_api + processor_id
        res = requests.get(url, auth=CLOUDERA_CREDENTIALS)
        return custom_response(res)

    @classmethod
    def update_run_status(cls, processor_id, clientId, state="STOPPED"):

        data = {
            "revision": {
                'clientId': clientId,
                'version': 1
            },
            "state": state,
            "disconnectedNodeAcknowledged": True
        }

        url = nifi_api + f"{processor_id}/run-status"

        res = requests.put(
            url,
            data=data,
            auth=CLOUDERA_CREDENTIALS,
            headers={'content-type': 'application/json'}
        )
        return custom_response(res)

## Connections
> NIFI REST API for Connections

In [ ]:
#export 
class Connection:
    
    nifi_api = NIFI_API_CONNECTIONS
    


    @classmethod
    def list_queues(cls, processor_id):
        url = cls.nifi_api + processor_id
        res = requests.get(url, auth=CLOUDERA_CREDENTIALS)
        return custom_response(res)
    
    

    @classmethod
    def update_run_status(cls, processor_id, clientId, state="STOPPED"):

        data = {
            "revision": {
                'clientId': clientId,
                'version': 1
            },
            "state": state,
            "disconnectedNodeAcknowledged": True
        }

        url = nifi_api + f"{processor_id}/run-status"

        res = requests.put(
            url,
            data=data,
            auth=CLOUDERA_CREDENTIALS,
            headers={'content-type': 'application/json'}
        )
        return custom_response(res)

# FlowFile Queues
> NIFI REST API for Flowfile Queues

In [ ]:
#export 
class Flowfile:
    
    nifi_api = NIFI_API_FLOWFILE_QUEUES
    
    @classmethod
    def listing_request(cls,connection_id):
        url = cls.nifi_api + f"{connection_id}/listing-requests"
        res = requests.post(url, auth=CLOUDERA_CREDENTIALS)
        return custom_response(res)

    @classmethod
    def list_queues(cls,connection_id):
        listing_request,_ = cls.listing_request(connection_id)
        listing_request_id  = listing_request["listingRequest"]['id']
        url = cls.nifi_api + f"{connection_id}/listing-requests/{listing_request_id}"
        res = requests.get(url, auth=CLOUDERA_CREDENTIALS)
        return custom_response(res)
        

In [ ]:
custom_response?

Signature: custom_response(res)
Docstring: <no docstring>
File:      /wd/nbs/nifi_api/tools.py
Type:      function


In [ ]:
Flowfile.listing_request("gonna fail")

('No JSON available', <Response [409]>)